In [1]:
import pandas as pd 

In [2]:
!dir db 

 Volume in drive C is Windows
 Volume Serial Number is 6EA7-3DCA

 Directory of C:\Users\p2p2l\projects\wgong\cs-faculty\app\db

04/29/2023  12:31 PM    <DIR>          .
04/29/2023  03:26 PM    <DIR>          ..
04/28/2023  08:20 AM             2,622 create_g_tables.sql
04/27/2023  06:28 AM               830 dev-copy.sql
04/29/2023  10:38 AM            12,144 dev.sql
04/29/2023  11:40 AM            18,710 faculty-CMU-CS.xlsx
04/29/2023  11:29 AM         9,973,760 faculty-Cornell-CS-20230422.duckdb
04/21/2023  11:17 PM         4,468,736 faculty-Cornell-CS.duckdb
04/29/2023  11:48 AM            34,554 faculty-Cornell-CS.xlsx
04/29/2023  11:41 AM            24,280 faculty-MIT-AID.xlsx
04/29/2023  11:42 AM            21,974 faculty-MIT-CS.xlsx
04/29/2023  11:42 AM            18,556 faculty-Stanford-CS.xlsx
04/29/2023  11:42 AM            26,743 faculty-UCB-CS.xlsx
04/29/2023  11:43 AM            20,183 faculty-UIUC-CS.xlsx
              12 File(s)     14,623,092 bytes
               2 Dir(

In [4]:
xlsx_obj = pd.ExcelFile("db/faculty-UCB-CS.xlsx", engine="openpyxl")

In [5]:
xlsx_obj.sheet_names

['Faculty', 'Research Groups']

In [20]:
data_dict= {}

for sheet in xlsx_obj.sheet_names:
    key = sheet.lower().replace(" ", "_")
    data_dict[key] = pd.read_excel(xlsx_obj, sheet, keep_default_na=False)

In [21]:
data_dict.keys()

dict_keys(['faculty', 'research_groups'])

In [22]:
from db import *

In [30]:
df_dict = {}
with DBConn() as _conn:
    for key in data_dict.keys():
        view_name = f"v_{key}"
        print(f"view_name = {view_name}")
        df = data_dict[key]
        print(df.columns)
        _conn.register(f"{view_name}", df)
        sql_stmt = ""
        if key == "faculty":
            sql_stmt = f"""insert into g_person (
                id, person_type, name, job_title, phd_univ, phd_year, research_area, research_concentration, 
                research_focus, url, img_url, phone, email, cell_phone, office_address, department, org
            )
            select 
                uuid() as id, 'faculty', name, job_title, phd_univ, phd_year, research_area, research_concentration, 
                research_focus, url, img_url, phone, email, cell_phone, office_address, department, org 
            from {view_name}"""
        elif key == "research_groups":
            sql_stmt = f"""insert into g_entity (
                id, entity_type, name, url
            )
            select 
                uuid() as id, 'research_group', research_group, url 
            from {view_name}"""
            
        if sql_stmt:
            print(f"sql_stmt =\n {sql_stmt}")
            df_dict[key] = _conn.execute(sql_stmt).df()

view_name = v_faculty
Index(['name', 'job_title', 'phd_univ', 'phd_year', 'research_area',
       'research_concentration', 'research_focus', 'url', 'img_url', 'phone',
       'email', 'cell_phone', 'office_address', 'department', 'org'],
      dtype='object')
sql_stmt =
 insert into g_person (
                id, person_type, name, job_title, phd_univ, phd_year, research_area, research_concentration, 
                research_focus, url, img_url, phone, email, cell_phone, office_address, department, org
            )
            select 
                uuid() as id, 'faculty', name, job_title, phd_univ, phd_year, research_area, research_concentration, 
                research_focus, url, img_url, phone, email, cell_phone, office_address, department, org 
            from v_faculty
view_name = v_research_groups
Index(['research_group', 'url'], dtype='object')
sql_stmt =
 insert into g_entity (
                id, entity_type, name, url
            )
            select 
               

In [31]:
df_dict

{'faculty':    Count
 0    131,
 'research_groups':    Count
 0     21}

In [32]:
with DBConn() as _conn:
    sql_stmt = """select  t.table_name
    from information_schema.tables t where t.table_name like 'g_%';"""
    df = pd.read_sql(sql_stmt, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_34644\2922519108.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_stmt, _conn)


In [34]:
df["table_name"].to_list()

['g_column_props', 'g_entity', 'g_note', 'g_person', 'g_work', 'g_relation']

In [35]:
from datetime import datetime

In [36]:
str(datetime.now())

'2023-04-29 17:18:37.741270'

In [37]:
datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

'2023-04-29_17-20-01'